# Analyzing and Visualizing Data from Foxglove Data Platform

**[Foxglove Data Platform](https://foxglove.dev/data-platform) is a scalable platform for organizing and managing your team's robotics data.** You can log in to [its web console interface](https://console.foxglove.dev) to upload data, tag events of interest, and query data for a given robot and time range, even if that data spans multiple recording sessions.

In this notebook, **we'll demonstrate how to retrieve messages from Data Platform and process them for insights.** We'll be using self-driving car data from the [nuScenes dataset](https://www.nuscenes.org/nuscenes), and writing Python code to visualize its route, IMU acceleration, and perceived objects.

In [ ]:
# install some dependencies
%pip install mcap-ros2-support==0.1.0 mcap-ros1-support==0.4.0 foxglove-data-platform==0.2.3 pandasql > /dev/null

### Loading data

The first step in our analysis is loading some data. We'll use the foxglove data platform client library.

In [ ]:
from foxglove_data_platform.client import Client

# Read-only public key for demonstration purposes
client = Client(token="fox_sk_1dbfnSNxWmtT4cV82YCMA3lbQE3hcE3E")

To query for data, we need to know a device id and some time range. The get_coverage method shows us available devices and data for 2018.

In [ ]:
from datetime import datetime
import pandas as pd

all_devices_coverage = client.get_coverage(start=datetime(2018, 1, 1), end=datetime(2019,1,1))
all_devices_coverage = sorted(all_devices_coverage, key=lambda c: c['start'])
pd.DataFrame(all_devices_coverage).head()

For this notebook we want to restrict our search to a single device. In this data platform organization, different devices record data using different message encodings. The Python client library handles message decoding for us, so we can use the same code to explore messages encoded in JSON, Protobuf, ROS1 and ROS2 (CDR).

In [ ]:
device_id = "dev_fj7b6mZSD54rJO7v"      # use this device ID to get ROS1-encoded messages.
# device_id = "dev_KphdijgpmmnnmYaH"    # use this device ID to get ROS2 CDR-encoded messages.
coverage = [r for r in all_devices_coverage if r["device_id"] == device_id]

We can inspect the topics and datatypes available for a given time range and device ID with `get_topics`.

In [ ]:
topics = client.get_topics(device_id=device_id, start=coverage[1]["start"], end=coverage[1]["end"])
pd.DataFrame(topics)

Fetch GPS messages for the first entry from our previous coverage request. We could fetch data across any start/end range and device.

We limit our data to the `/gps` topic since that's all we need for our analysis.

In [ ]:
gps_messages = [
    (message.latitude, message.longitude)
    for topic, record, message in client.get_messages(
        device_id=device_id,
        start=coverage[1]["start"],
        end=coverage[1]["end"],
        topics=["/gps"],
    )
]
pd.DataFrame(gps_messages, columns=["lat", "lon"]).head()

We now know the basics to loading our data. Next, we can analyze the data using existing jupyter notebook tools and practices.

### Mapping the route

Let's see the route that our car took by plotting its GPS coordinates on a map.

We'll load data from on August 1, 2018 (`coverage[1]`) and use list comprehension to convert our messages into a list of tuples that we can insert into a `pandas` dataframe.

In [ ]:
gps_messages = [
    (message.latitude, message.longitude)
    for topic, record, message in client.get_messages(
        device_id=device_id,
        start=coverage[1]["start"],
        end=coverage[1]["end"],
        topics=["/gps"],
    )
]

**TIP**: We recommend splitting your data fetching and processing into separate cells. This lets you iterate on your analysis without re-downloading the data.

In [ ]:
import folium

figure = folium.Figure(width=640, height=480)
map = folium.Map(location=gps_messages[0], zoom_start=200, width="100%")
folium.PolyLine(
    locations=gps_messages,
    weight=10,
    color="purple",
).add_to(map)
map.add_to(figure)

We can see that on August 1, 2018, our self-driving car navigated a stretch of Congress Street in Boston.

### Plotting IMU acceleration

For our first analysis, we focused on just one recorded drive. For this analysis, let's fetch messages across a longer time range to plot our robot's acceleration across all 2018 drives.

We can take advantage of Data Platform's ability to fetch messages across multiple recording sessions by specifying the time range we want in our `get_messages` call.

In [ ]:
imu_messages = [
    {
        "time": pd.Timestamp(message.header.stamp.to_nsec(), unit="ns").isoformat(),
        "accel_x": message.linear_acceleration.x,
        "accel_y": message.linear_acceleration.y,
    }
    for topic, record, message in client.get_messages(
        device_id=device_id,
        start=coverage[0]["start"],
        end=coverage[-1]["end"],
        topics=["/imu"],
    )
]

In [ ]:
pd.DataFrame(imu_messages).plot(x="time", figsize=(10, 6), rot=45);

From the output above, we can see how our robot's x and y acceleration fluctuated throughout its 2018 drives.

### Classifying perceived object markers

Finally, let's classify the perceived object markers that our self-driving car published while on the road.

We'll again use one specific time range – `coverage[1]` – and query its `/markers/annotations` topic messages.

In [ ]:
marker_messages = client.get_messages(
    device_id=device_id,
    start=coverage[1]["start"],
    end=coverage[1]["end"],
    topics=["/markers/annotations"],
)

In our dataset, each marker color coorresponds to a specific _classification_. We'll use the marker color to lookup the classification and group markers by classification.

In [ ]:
import matplotlib as mpl
from pandasql import sqldf

color_to_classname = {
    "#000000": "noise",
    "#468250": "animal",
    "#0000e6": "human.pedestrian.adult",
    "#87ceeb": "human.pedestrian.child",
    "#f08080": "human.pedestrian.construction_worker",
    "#db7093": "human.pedestrian.personal_mobility",
    "#000080": "human.pedestrian.police_officer",
    "#f08080": "human.pedestrian.stroller",
    "#8a2be2": "human.pedestrian.wheelchair",
    "#708090": "movable_object.barrier",
    "#d2691e": "movable_object.debris",
    "#696969": "movable_object.pushable_pullable",
    "#2f4f4f": "movable_object.trafficcone",
    "#bc8f8f": "static_object.bicycle_rack",
    "#dc143c": "vehicle.bicycle",
    "#ff7f50": "vehicle.bus.bendy",
    "#ff4500": "vehicle.bus.rigid",
    "#ff9e00": "vehicle.car",
    "#e99646": "vehicle.construction",
    "#ffd700": "vehicle.emergency.ambulance",
    "#ffd700": "vehicle.emergency.police",
    "#ff3d63": "vehicle.motorcycle",
    "#ff8c00": "vehicle.trailer",
    "#ff6347": "vehicle.truck",
    "#00cfbf": "flat.driveable_surface",
    "#af004b": "flat.other",
    "#4b004b": "flat.sidewalk",
    "#70b43c": "flat.terrain",
    "#deb887": "static.manmade",
    "#ffe4c4": "static.other",
    "#00af00": "static.vegetation",
    "#fff0f5": "vehicle.ego",
}

flattened_markers = []
for topic, record, message in marker_messages:
    for marker in message.markers:
        color = mpl.colors.to_hex([marker.color.r, marker.color.g, marker.color.b])
        class_name = color_to_classname[color]
        flattened_markers.append((marker.text, class_name))
annotations = pd.DataFrame(flattened_markers, columns=["annotation_id", "class_name"])

pysqldf = lambda q: sqldf(q, globals())
res = pysqldf(
    "SELECT class_name,COUNT(*) as count FROM annotations GROUP BY class_name ORDER BY count DESC"
)

res

In [ ]:
res.plot.bar(x="class_name", y="count", legend=False);

We can see how many examples of each perceived object our self-driving car encountered (943 cars, 879 adult pedestrians, etc.).

### End

This demo illustrated some of the many ways you can analyze your robotics data wit Jupyter notebooks and [Foxglove Data Platform](https://foxglove.dev/data-platform).

Signup and analyze your data at https://console.foxglove.dev/signin

Join the Foxglove [Slack community](https://foxglove.dev/slack) and follow [our blog](https://foxglove.dev/blog) for more ideas on how to integrate Data Platform into your robotics development workflows.